## Analysis of Reduced_gg meteo files

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
from pyresample import bilinear, geometry, kd_tree
from pyresample import utils
import xarray as xr

In [ ]:
%matplotlib notebook

In [ ]:
filename= '/Users/brey/DATA/20130301.00.tropical_cyclone.grib'

### With pygrib

In [ ]:
import pygrib

In [ ]:
d=pygrib.open(filename)

In [ ]:
for m in d:
    print m

In [ ]:
msl = d.message(1)

In [ ]:
msl.keys()

In [ ]:
for key in msl.keys():
    print key, msl[key]

### with eccodes

In [ ]:
from gribapi import *

In [ ]:
f=open(filename)
gid = grib_new_from_file(f)

In [ ]:
iterid = grib_keys_iterator_new(gid,'ls')
while grib_keys_iterator_next(iterid):
    keyname = grib_keys_iterator_get_name(iterid)
    keyval = grib_get_string(iterid,keyname)
    print "%s = %s" % (keyname,keyval)
    
grib_keys_iterator_delete(iterid)
grib_release(gid)
 
f.close()


In [ ]:
keys={'Ni',
'Nj',
'iDirectionIncrementInDegrees',
'iScansNegatively',
'jPointsAreConsecutive',
'jScansPositively',
'latitudeOfFirstGridPointInDegrees',
'latitudeOfLastGridPointInDegrees',
'longitudeOfFirstGridPointInDegrees',
'longitudeOfLastGridPointInDegrees'
     }

In [ ]:
f=open(filename)
gid = grib_new_from_file(f)
for key in keys:
        if not grib_is_defined(gid, key):
            raise ValueError("Key '%s' was not defined" % key)
        print('%s=%s' % (key, grib_get(gid, key)))
 


In [ ]:
name=grib_get(gid, 'shortName')
mv=grib_get(gid,'missingValue')

date=grib_get(gid, 'date')
dataTime=grib_get(gid, 'dataTime')
stepRange=grib_get(gid, 'stepRange')

lonfgp=grib_get(gid,'longitudeOfFirstGridPointInDegrees')
latfgp=grib_get(gid,'latitudeOfFirstGridPointInDegrees')
lonlgp=grib_get(gid,'longitudeOfLastGridPointInDegrees')
latlgp=grib_get(gid,'latitudeOfLastGridPointInDegrees')
Ni=grib_get(gid,'Ni')
Nj=grib_get(gid,'Nj')


values=grib_get_values(gid)
elat=grib_get_array(gid,'latitudes')
elon=grib_get_array(gid,'longitudes')

### Compute the regular lat/lon 

In [ ]:
ss=grib_get_array(gid,'pl')  # lons per lat for the reduced_gg grid
nlats=ss.size

In [ ]:

nlons = 2*nlats
delon = 360./nlons
lons = np.linspace(lonfgp,lonlgp,nlons)
# compute gaussian lats (north to south)
lats = pygrib.gaulats(nlats)
if latfgp > latlgp :
    lats = lats[::-1]
elon,elat = np.meshgrid(lons,lats) # make 2-d arrays

### Use redtoreg 

In [ ]:
from redtoreg import _redtoreg
ny=2*np.size(ss)

dat=_redtoreg(ny,ss,values,mv)
q=np.flipud(dat)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(elon,elat,q, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

## Resample on rectangular grid

In [ ]:
minlon, maxlon = elon.min(), elon.max()

In [ ]:
minlat, maxlat = elat.min(), elat.max()

In [ ]:
import pyproj
prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
a0,a1= prj(-180., minlat)
a2, a3 = prj(180., maxlat)
a0,a1,a2,a3

In [ ]:
Nj,Ni = elon.shape
Nj,Ni

In [ ]:
area_id = 'Global'
description = 'ECMWF OPER'
proj_id = 'eqc'
projection = '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
x_size = Ni
y_size = Nj
area_extent = (a0, a1, a2, a3)
target_def = utils.get_area_def(area_id, description, proj_id, projection,
                                       x_size, y_size, area_extent)
print(target_def)


In [ ]:
plons,plats = geometry.AreaDefinition.get_lonlats(target_def)

In [ ]:
from pyresample.utils import check_and_wrap
slons,slats  = check_and_wrap(elon,elat)

In [ ]:
orig = geometry.SwathDefinition(lons=slons,lats=slats) # original points

#### Nearest

In [ ]:
#kd_tree.resample_nearest(orig,q,targ,radius_of_influence=50000,fill_value=999999)
near = kd_tree.resample_nearest(orig, q, target_def, radius_of_influence=50000, epsilon=0.5)

In [ ]:
#w=np.ma.masked_array(iu10,iu10==999999.)

In [ ]:
xv=[q,near]
xp=[elon,plons]
yp=[elat,plats]

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
                  dict(map_projection=projection))

fig = plt.figure(figsize=(12,8))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                    nrows_ncols=(1, 2),
                    axes_pad=1,
                    cbar_location='right',
                    cbar_mode='single',
                    cbar_pad=0.5,
                    cbar_size='10%',
                    label_mode='')  # note the empty label_mode


for i, ax in enumerate(axgr):
        ax.coastlines()
        ax.set_xticks(np.linspace(-180, 180, 5), crs=projection)
        ax.set_yticks(np.linspace(-90, 90, 5), crs=projection)
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)

        p = ax.contourf(xp[i], yp[i],xv[i],
                        transform=projection)#,
                     #   cmap='RdBu')

axgr.cbar_axes[0].colorbar(p)




#### bilinear

In [ ]:
blinear = bilinear.resample_bilinear(q, orig, target_def,
                                      radius=50e3, neighbours=32,
                                      nprocs=4, fill_value=None,
                                      reduce_data=False, segments=None,
                                      epsilon=0)

#### compare Nearest with Bilinear

In [ ]:
df = np.abs(blinear-near)

In [ ]:
df.max()

In [ ]:
np.argwhere(df==df.max())

In [ ]:
plt.figure()
plt.contour(np.flipud(df))
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(plons,plats,df, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

#### compare with original method

In [ ]:
df = np.abs(q-blinear)

In [ ]:
np.argwhere(df==df.max())

In [ ]:
plt.figure()
plt.contour(np.flipud(df))
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(plons,plats,df, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();